# Hyperbolic trajectory
----

In [1]:
import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.linalg
from System import MakeSystem, FindHyperbolicTrajectory, MakeLinearisedSolution

In [2]:
%matplotlib qt
x = np.linspace(-5,5,100)
y = -np.sign(x)*(1-1/(np.cosh(x)**2))**.5
plt.plot(x,y)
plt.xlabel('q')
plt.ylabel('dq/dt')
plt.show() 

In [3]:
x = np.linspace(-3,3,100)
y = 1/(np.cosh(x)**2)
plt.plot(x,y)
plt.xlabel('x')
plt.ylabel('sech^2(x)')
plt.show()

In [4]:
k = 1
ForcingParameters = array([.1,.7,-.15,2])
f,lminus,lplus,G = MakeSystem(k,ForcingParameters) # system in stable/unstable coordinates

XQ = MakeLinearisedSolution(ForcingParameters,lplus,lminus)
tmin,tmax = -2,10

N = 201
Xhyp,t,X0 = FindHyperbolicTrajectory(f,lminus,lplus,XQ,tmin,tmax,N)

 after 6 iterations an estimate with error of F =5.924255865251801e-12 and delta = 1.0727648856633993e-09was produced


In [5]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')

ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory, quasi-periodic")
ax.plot( X0[:,0], X0[:,1],t,linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory, quasi-periodic")
plt.legend(fontsize=14)
ax.set_xlabel("$x$", fontsize=14)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
ax.set_ylabel("$y$", fontsize=14)
ax.set_zlabel("$t$", fontsize=14)
plt.show()

plt.plot(t, Xhyp[:,0], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,0], color='red',  linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory")
plt.title("Hyperbolic trajectories $x$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$x$", fontsize=18)
plt.legend(fontsize=6)
plt.show()
plt.plot(t, Xhyp[:,1], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,1], color='red',  linestyle='--', linewidth=2, marker='x', label="Linearised hyperbolic trajectory")
plt.title("Hyperbolic trajectories $y$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$y$", fontsize=18)
plt.legend(fontsize=6)
plt.show()

# Stable manifold data collection
---

In [6]:
from StableManifoldTrajectories import FindStableTrajectory,FindUnstableTrajectory
import scipy.io
import scipy.interpolate
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import Rbf
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interp1d


In [7]:
from numpy import nan
def ySM(p,s):
    for l in range(0,len(tindices)):
        if t[tindices[l]] == s:
            return  interpyS[t[tindices[l]]](array([p]))
        elif t[l] < s and t[l+1]> s:
            t0 = t[tindices[l]] 
            t1 = t[tindices[l+1]]
            break
    if l< len(tindices) - 2:
        SM1 = interpyS[t0](array([p]))
        SM2 = interpyS[t1](array([p]))
        return SM1 + ((s - t0)/(t1-t0))*(SM2-SM1)
    else: 
        return nan
def xUM(q,s):
    for l in range(0,len(tindices)):
        if t[tindices[l]] == s:
            return  interpxU[t[tindices[l]]](array([q]))
        elif t[l] < s and t[l+1]> s:
            t0 = t[tindices[l]] 
            t1 = t[tindices[l+1]]
            break
    if l< len(tindices) - 2:
        UM1 = interpxU[t0](array([q]))
        UM2 = interpxU[t1](array([q]))
        return UM1 + ((s - t0)/(t1-t0))*(UM2-UM1)
    else: 
        return nan
    

In [8]:
def makeinterpyS(P,yvals,tindices):
    interpyS = {  t[tindices[0]]: interp1d(P,yvals[:len(P)] )}
    for l in range(1,len(tindices)):
        interpyS[t[tindices[l]]] = interp1d(P,yvals[l*len(P):(l+1)*len(P)],kind = 'linear')
    return interpyS

def makeinterpxU(Q,xvals,tindices):
    interpxU = {  t[tindices[0]]: interp1d(Q,xvals[:len(Q)] )}
    for l in range(1,len(tindices)):
        interpxU[t[tindices[l]]] = interp1d(Q,xvals[l*len(Q):(l+1)*len(Q)],kind = 'linear')
    return interpxU

In [9]:
P = np.linspace(-2,2,10)
Q =np.linspace(-2,2,10)
tmin,tmax = -3,3
t = np.linspace(tmin,tmax,N)
N = 200
trangestart = N//5
trangeend = 4*trangestart
tindices = np.arange(int(N//5),int(4*N//5),10)
#tindices = np.array([3*N//4])
trange = t[tindices]
yvals = np.zeros(len(P)*len(tindices))
xvals = np.zeros(len(Q)*len(tindices))

l=0
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--', color = 'red', linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
for J in tindices:
    for i in range(0,len(P)):
        p = P[i]
        X,tnew,X0,Jnew = FindStableTrajectory(f,lminus,lplus,tmin,tmax,N,p,J,Xhyp)
       # if l==0:
        #    ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o', label="Non-linear Stable manifold trajectory")
        #else:
        #    ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o')
        yvals[l] = X[Jnew][1]   # output of function applied to (trange[J],P[i])
        q = Q[i]
        X,tnew,X0,Jnew = FindUnstableTrajectory(f,lminus,lplus,tmin,tmax,N,q,J,Xhyp)
     #   if l==0:
    #        ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o', label="Non-linear Stable manifold trajectory")
     #   else:
     #       ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o')
        xvals[l] = X[Jnew][0]   # output of function applied to (trange[J],P[i])
        l+=1
        if int(100*l/(len(tindices)*len(P)))%5 ==0:
            print(str(int(100*l/(len(tindices)*len(P)))) + "% complete")
        

 after 24 iterations an estimate with error of F =2.633939567414213e-09 and delta = 7.366453723323049e-08was produced
 after 28 iterations an estimate with error of F =5.289178400954229e-09 and delta = 9.671135871139307e-08was produced
0% complete
 after 21 iterations an estimate with error of F =2.8084557811309096e-09 and delta = 8.093641692686972e-08was produced
 after 31 iterations an estimate with error of F =5.711289248910326e-09 and delta = 9.587842139557169e-08was produced
 after 17 iterations an estimate with error of F =2.189813718537609e-09 and delta = 6.518777734909285e-08was produced
 after 38 iterations an estimate with error of F =6.682459590110343e-09 and delta = 9.945214869622429e-08was produced
 after 12 iterations an estimate with error of F =7.541424675860952e-10 and delta = 2.2089976834186204e-08was produced
 after 22 iterations an estimate with error of F =4.049788698918223e-09 and delta = 8.821818043052082e-08was produced
 after 7 iterations an estimate with error

In [10]:
interpyS = makeinterpyS(P,yvals,tindices)
interpxU = makeinterpxU(Q,xvals,tindices)

Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'green', linewidth=3, marker='x')
    #plt.legend(fontsize=14)
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
    ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'blue', linewidth=3, marker='x')
    
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()
    
    
    

In [11]:
np.save("P.npy",P)
np.save("yvals.npy",yvals)
np.save("tindices.npy",tindices)
np.save("t.npy",t)
np.save("Xhyp.npy",Xhyp)
np.save("Q.npy",Q)
np.save("xvals.npy",xvals)
np.save("ForcingParameters.npy",ForcingParameters)

In [12]:
Q =np.linspace(-2,2,10)
N = 200

trangestart = N//5
trangeend = 4*trangestart
#tindices = np.arange(int(N//5),int(4*N//5),10)
#tindices = np.array([3*N//4])
trange = t[tindices]
P = np.load("P.npy")
Q = np.load("Q.npy")
yvals = np.load("yvals.npy")
xvals = np.load("xvals.npy")
ForcingParameters = np.load("ForcingParameters.npy")
tindices = np.load("tindices.npy")
t = np.load("t.npy")
Xhyp = np.load("Xhyp.npy")

## Plots
----

Manifolds in original coordinates.

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Original[:,0], Original[:,1], (lplus-lminus)*t,linestyle='--', color = 'green', linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    if J == tindices[0]:
        ax.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0],M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1] ,(lplus-lminus)*t[J], color = 'blue', linewidth=3, marker='x',label = 'Stable manifold')
    else:
        ax.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0],M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1] ,(lplus-lminus)*t[J], color = 'blue', linewidth=3, marker='x')
    
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
    if J ==tindices[0]:
        ax.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1],(lplus-lminus)*t[J], color = 'red', linewidth=3, marker='x',label = "Unstable manifold")
    else:
        ax.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1],(lplus-lminus)*t[J], color = 'red', linewidth=3, marker='x')
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    plt.legend()
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()

Time slice in original coordinates.

In [13]:

#Changing coordinates back to original
Minv = (lplus - lminus)**(-1)*np.array([[lplus,-1],[-lminus,1]])
M = np.linalg.inv(Minv)
fig = plt.figure(figsize=(15,15))
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
J= 100
Yvals =0*Pinterp
Original = 0*Xhyp

for i in range(0,Original.shape[0]):
    Original[i] = M.dot(Xhyp[i])

for i in range(0,len(Pinterp)):
    Yvals[i] = ySM(Pinterp[i],t[J])
    
# changing coords    
    
plt.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0], M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1]   ,         color = 'blue', linewidth=3, marker='x', label = "Stable manifold")
plt.plot(Original[J,0],Original[J,1],marker = 'o',label ="hyp traj",color = 'green')
for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
plt.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1], color = 'red', linewidth=3, marker='x',label = 'Unstable manifold')
    
plt.legend(fontsize=14)
plt.xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=14)
plt.show()

# Heat map
----


In [14]:
xhyp = lambda s: np.interp(s,(lplus-lminus)*t,Original[:,0])
yhyp = lambda s: np.interp(s,(lplus-lminus)*t,Original[:,1])

In [ ]:
# functions to determine if it passes over
f = lambda t,y : np.array([y[1], np.tanh(y[0])*np.cosh(y[0])**(-2) - k*y[1]]+(lplus-lminus)*G((lplus-lminus)*t)) # vector field in original coordinates
#fig = plt.figure(figsize=(15,15))
def event(s,y):
    #return (Minv.dot(y) - np.array([xhyp(s),yhyp(s)]))[0]
    return y[0]- xhyp(s)
fig = plt.figure(figsize=(15,15))
event.terminal = True
# functions to determine if it passes over
def passOver(y0):
    res = solve_ivp (f, [0,100], y0 , method = "RK45" , events=[event],max_step =0.05 , rtol = 1e-8 )
    y = res .y. transpose()
    t = res .t  
    s = t[-1]
    #plt.plot(y[:,0],y[:,1])
    if s<100 and y[-1][1] > yhyp(s):  # the trajectories which move left to right
#        print(str(y0) + "left to right")
#        plt.plot(y0[0],y0[1], marker = 'o',color = "red")
        return 1
    elif s<100 and y[-1][1] < yhyp(s):  # the trajectories which move right to left
#        print(str(y0) + "right to left")
#        plt.plot(y0[0],y0[1],marker = 'o', color = "blue")
        return 0
    elif y[-1][0]- xhyp(s) > 0 and  y0[0]- xhyp(0)>0 and s == 100: # trajectories which stay on right
#        print(str(y0) + "right to right")
#        plt.plot(y0[0],y0[1],marker = 'o', color = "brown")
        return 1
    elif y[-1][0]- xhyp(s) < 0 and  y0[0]- xhyp(0)<0 and s == 100: # trajectories which stay on left
#        print(str(y0) + "left to left")
#        plt.plot(y0[0],y0[1],marker = 'o', color = "green")
        return 0
    else:
        return 0


#creating and storing the data.

x = np.meshgrid(np.linspace(-3, 3,100) , np.linspace(-3,3,100),indexing= 'ij')
z = np.zeros((np.shape(x)[1],np.shape(x)[2]))
for i in range(0,np.shape(x)[1]):
    for j in range(0,np.shape(x)[2]):
        y0 = np.array([x[0][i,j],x[1][i,j]])
        z[i,j] = passOver(y0)
plt.show
np.save("initial conditions.npy" , x)
np.save("Points moving over.npy", z)

In [ ]:
fig = plt.figure(figsize=(15,15)) 
plt.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0], M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1]   ,         color = 'blue', linewidth=3, marker='x', label = "Stable manifold")
plt.plot(Original[J,0],Original[J,1],marker = 'o',label ="hyp traj",color = 'green')
for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
plt.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1], color = 'red', linewidth=3, marker='x',label = 'Unstable manifold')
    
plt.legend(fontsize=14)
plt.xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=14)
plt.show()
plt.contourf(x[0][:],x[1][:],z)
plt.colorbar()
plt.xlabel("x", fontsize=14)
plt.ylabel("y", fontsize=14)
plt.show()

## Old plots
----

Plot of  manifolds in stable/unstable coordinates.

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--', color = 'green', linewidth=3, marker='o', label="Hyperbolic trajectory")
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    if J == tindices[0]:
        ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'blue', linewidth=3, marker='x',label = "Stable manifold")
    else:
        ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'blue', linewidth=3, marker='x')
    
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
        
    if J == tindices[0]:
        ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x', label = "Unstable manifold")
    else:
        ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x')
    
    plt.legend(fontsize = 14)
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()

Time slice in stable/unstable coordinates

In [ ]:
fig = plt.figure(figsize=(15,15))
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
J= 100
Yvals =0*Pinterp
for i in range(0,len(Pinterp)):
    Yvals[i] = ySM(Pinterp[i],t[J])
plt.plot(Pinterp[:]+Xhyp[J][0],Yvals[:], color = 'blue', linewidth=3, marker='x', label = "stable manifold")
plt.plot(Xhyp[J][0],Xhyp[J][1],marker = 'o',label ="hyp traj",color = 'green')
for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
plt.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x',label = 'unstable manifold')
    
plt.legend(fontsize=14)
plt.xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=14)
plt.show()